In [1]:
# Import all required libraries  
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adamax
import numpy as np
from numpy import genfromtxt

Using TensorFlow backend.


In [2]:
np.random.seed(1337)

# data from dataset 
X_train = genfromtxt('train_data.csv', delimiter=',')
y_train = genfromtxt('train_target.csv', delimiter=',')
X_test = genfromtxt('test_data.csv', delimiter=',')

In [3]:
# We have to reshape the input to fit into the Tensorflow 
X_train = X_train.reshape(X_train.shape[0], 48, 48,1)
X_test = X_test.reshape(X_test.shape[0], 48, 48,1 )

# Typecast the pixels of test and train
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Divide the data with the maximum color value i.e. 255
X_train = X_train/ 255.0
X_test = X_test/255.0

Y_train = np_utils.to_categorical(y_train, 3)

#VGG model added batch normalization after convolutional layer.

model = Sequential()

# Add layers of zeropadding, convolutional layer, and maxPooling (batch normalisation on each layer)
model.add(ZeroPadding2D((1, 1), input_shape=(48,48,1)))
model.add(Convolution2D(100, (3, 3), strides=(1, 1), padding ="same", activation='relu')) 
model.add (ZeroPadding2D((1,1)))
model.add(Convolution2D(100, (3, 3), strides=(1, 1), padding ="same", activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2), dim_ordering="tf"))
model.add (ZeroPadding2D((1,1)))

model.add(Convolution2D(256, (3, 3), strides=(1, 1), padding ="same", activation='relu')) 
model.add (ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), strides=(1, 1), padding ="same", activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2), dim_ordering="tf"))
model.add (ZeroPadding2D((1,1)))

model.add(Convolution2D(512, (3, 3), strides=(1, 1), padding ="same", activation='relu')) 
model.add (ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), strides=(1, 1), padding ="same", activation='relu')) 
model.add (ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), strides=(1, 1), padding ="same", activation='relu')) 
model.add (ZeroPadding2D((1,1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2), dim_ordering="tf"))

# Finally add the fully connected layers
model.add(Flatten())

model.add(Dense(512, activation='relu')) 
model.add(BatchNormalization())
model.add(Dropout(0.8))

model.add(Dense(512, activation='relu')) 
model.add(BatchNormalization())
model.add(Dropout(0.8)) 

model.add(Dense(3, activation='softmax'))

# Compile the model with optimization
adam = Adamax(lr=0.0015, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam , metrics=['accuracy'])

# Train the model using train_data
model.fit(X_train, Y_train, batch_size=128, epochs=100, validation_split=0.15) # batch = 100 split= 0.10

# Predicting the results based on learning
predictions = model.predict(X_test)
#Convert prediction from probabilites in vectors ([.11,.22,.5]) to scalar integers ([0],[1],[2])
output = predictions.argmax(axis=1)


C:\Users\Ashish\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_last")`
C:\Users\Ashish\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_last")`
C:\Users\Ashish\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_last")`


Train on 10352 samples, validate on 2588 samples
Epoch 1/20
10352/10352 [==============================] - 1108s 107ms/step - loss: 2.7068 - acc: 0.4452 - val_loss: 1.5950 - val_acc: 0.5703
Epoch 2/20
10352/10352 [==============================] - 1318s 127ms/step - loss: 1.6053 - acc: 0.5299 - val_loss: 0.9137 - val_acc: 0.6159
Epoch 3/20
10352/10352 [==============================] - 1068s 103ms/step - loss: 1.2539 - acc: 0.5862 - val_loss: 0.8901 - val_acc: 0.6221
Epoch 4/20
10352/10352 [==============================] - 1070s 103ms/step - loss: 1.1043 - acc: 0.6242 - val_loss: 0.8163 - val_acc: 0.6406
Epoch 5/20
10352/10352 [==============================] - 1063s 103ms/step - loss: 1.0051 - acc: 0.6492 - val_loss: 0.7676 - val_acc: 0.6754
Epoch 6/20
10352/10352 [==============================] - 1068s 103ms/step - loss: 0.9399 - acc: 0.6753 - val_loss: 0.9266 - val_acc: 0.6716
Epoch 7/20
10352/10352 [==============================] - 1070s 103ms/step - loss: 0.8905 - acc: 0.6912 -

In [4]:
import csv
# Writes prediction to a csv file
def write_csv(ar, file_name="Result"):
    with open(file_name+".csv", 'w') as csvfile:
        writer = csv.writer(csvfile,delimiter=',',quoting = csv.QUOTE_NONE)
        row = ['Id','Category']
        writer.writerow(row)
        for i in range(ar.shape[0]):
            row = [i,ar[i]]  
            writer.writerow(row)
            i+=1
            
#writing prediction into a csv file            
write_csv(output)
print("Successfully created the prediction csv file.....!!")

Successfully created the prediction csv file.....!!
